# Gravity Density Model - Interactive Lab

<!--<badge>--><a href="https://colab.research.google.com/github/inlab-geo/cofi-examples/blob/main/notebooks/gravity/gravity_density_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><!--</badge>-->

> If you are running this notebook locally, make sure you've followed [steps here](https://github.com/inlab-geo/cofi-examples#run-the-examples-with-cofi-locally)
to set up the environment. (This [environment.yml](https://github.com/inlab-geo/cofi-examples/blob/main/environment.yml) file
specifies a list of packages required to run the notebooks)

Adapted from [gravity forward code](https://github.com/inlab-geo/inversion-test-problems/blob/main/contrib/gravityforward/__init__.py) written in inversion-test-problems

---

In [1]:
# -------------------------------------------------------- #
#                                                          #
#     Uncomment below to set up environment on "colab"     #
#                                                          #
# -------------------------------------------------------- #

# !pip install -U cofi

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from cofi import BaseProblem, InversionOptions, Inversion

from gravity_density_lib import *

np.random.seed(42)

In [3]:
# ---------------------------------------------------------------- #
#                                                                  #
#     Auxiliary code for problem definition, no need to modify     #
#                                                                  #
# ---------------------------------------------------------------- #

# Load true model and starting guesses
rec_coords, _, _, z_nodes, model = load_gravity_model()
Starting_model1, Starting_model2, Starting_model3 = load_starting_models()

# Create "observed" data by adding noise to forward solution
noise_level=0.05
gz = forward(model)
dataZ_obs= gz + np.random.normal(loc=0,scale=noise_level*np.max(np.abs(gz)),size=np.shape(gz))  

# Create jacobian
Jz = get_jacobian(model)

# Define depth weighting values
z0=18.6
beta=2
# Define regularization parameter
epsilon=0.2

# Create regularization
# Calculate depth weighting fcn - high values at low z, low values at high z, no zeros.
# Model is [Nx1] with N: no. of cells; W is [NxN] with weighting values on diagonal
W=depth_weight(z_nodes[:,0],z0,beta)
W=np.diag(W)

# Set CoFI problem:
grav_problem = BaseProblem()
grav_problem.name = "Gravity"
grav_problem.set_data(gz)

# Here I linked the function, not the result
grav_problem.set_forward(forward)

# Here I linked to the actual jacobian. Jacobian size is (MxN) with M: receiver and N: model cells
grav_problem.set_jacobian(Jz)

# Set regularization; reg is a function that takes the model as input
grav_problem.set_regularisation(reg_l1, epsilon, args=[W])

# Use default L2 misfit
grav_problem.set_data_misfit("L2")
grav_problem.set_initial_model(Starting_model3)

# Set gradient, in hope of helping optimisers converge better
def data_misfit_gradient(model):
    return 2* Jz.T @ (forward(model) - gz) / gz.shape[0]
grav_problem.set_gradient(lambda m: data_misfit_gradient(m) + epsilon*reg_gradient_l1(m, W))

In [4]:
# ---------------------------------------------------------------- #
#                                                                  #
#     Auxiliary code for widgets displaying, no need to modify     #
#                                                                  #
# ---------------------------------------------------------------- #

import json
from contextlib import contextmanager
from cofi.solvers import solvers_table

# clear optional parameters
grav_problem.set_initial_model(None)
grav_problem.set_regularisation(reg_l1, args=[W])
def depth_weight(z,z0,beta):
    return ((-z+z0)**(-beta/2))
beta=2

def adjust_problem(initial_model, z0, regularisation_factor):
    grav_problem.set_initial_model(initial_model)
    W=depth_weight(z_nodes[:,0],z0,beta)
    print("Weighting matrix", W)
    W=np.diag(W)
    reg_l1 = lambda model: np.linalg.norm(W @ model, 1)
    grav_problem.set_regularisation(reg_l1, regularisation_factor, args=[W])
    grav_problem.set_gradient(lambda m: data_misfit_gradient(m) + regularisation_factor*reg_gradient_l1(m, W))
    return grav_problem

def adjust_options(solving_method, tool, solver_params):
    inv_options = InversionOptions()
    inv_options.set_solving_method(solving_method)
    inv_options.set_tool(tool)
    inv_options.set_params(**solver_params)
    return inv_options

def plot_from_result(inv_result):
    result_model = inv_result.model.reshape(12,12,12)
    plt.imshow(result_model[::-1,5,:])
    plt.colorbar()

tmp_model = np.array(Starting_model1)
tmp_model[tmp_model>0] = 10
initial_models_table = {
    "Starting_model1": Starting_model1,
    "Starting_model2": Starting_model2,
    "Starting_model1(1000->10)": tmp_model,
    "all zeros": np.zeros((12**3)),
    "all ones": np.ones((12**3)),
}

def inversion(initial_model_id, z0, reg_factor, method, tool, solver_params):
    initial_model = initial_models_table[initial_model_id]
    grav_problem = adjust_problem(initial_model, z0, reg_factor)
    inv_options = adjust_options(method, tool, solver_params)
    inv_runner = Inversion(grav_problem, inv_options)
    result = inv_runner.run()
    # plot_from_result(result)
    return result

# inversion("Starting_model1", 1, "optimisation", "scipy.optimize.least_squares")

In [5]:
# -------------------------------------------------------------------------------------- #
#                                                                                        #
# Auxiliary code for widgets displaying, no need to modify                               #
#                                                                                        #
# Run this cell and start interacting :)                                                 #
#                                                                                        #
# If you have trouble displaying the interactive widgets locally, check advice here:     #
# https://stackoverflow.com/questions/36351109/ipython-notebook-ipywidgets-does-not-show #
#                                                                                        #
# -------------------------------------------------------------------------------------- #

from ipywidgets import interactive
import ipywidgets as widgets

method_widget = widgets.ToggleButtons(options=solvers_table.keys(), description="method")
tool_widget = widgets.RadioButtons(options=solvers_table["optimisation"].keys(), description="tool")
initial_model_widget = widgets.RadioButtons(options=initial_models_table.keys(), value="all ones", description="intial_model")
z0_widget = widgets.FloatSlider(min=0, max=17, value=3, description="z0")
reg_factor_widget = widgets.FloatLogSlider(base=10,value=0.0001,min=-15,max=1,step=0.2, description="reg_factor")
solver_specific_params_widget = widgets.VBox(children=[])
run_widget = widgets.Button(description="Run Inversion")

def update_params_widgets(tool_solver):
    params_children = []
    required_in_options = tool_solver.required_in_options
    for param in required_in_options:
        params_children.append(widgets.Text(value="", placeholder="required", description=param))
    optional_in_options = tool_solver.optional_in_options
    for param, dft in optional_in_options.items():
        params_children.append(widgets.Text(value=str(dft), placeholder="optional", description=param))
    solver_specific_params_widget.children = params_children

def method_updated(*args):
    tool_widget.options = solvers_table[method_widget.value].keys()
method_widget.observe(method_updated, 'value')

def tool_updated(*args):
    tool = tool_widget.value
    tool_solver = solvers_table[method_widget.value][tool]
    required_in_problem = tool_solver.required_in_problem
    initial_model_widget.layout.visibility = "visible" if "initial_model" in required_in_problem else "hidden"
    reg_factor_widget.layout.visibility = "visible" if "objective" in required_in_problem else "hidden"
    update_params_widgets(tool_solver)
tool_widget.observe(tool_updated, 'value')
update_params_widgets(solvers_table["optimisation"]["scipy.optimize.minimize"])

def button_on_click(*args): 
    method = method_widget.value
    tool = tool_widget.value
    initial_model_id = initial_model_widget.value
    z0 = z0_widget.value
    reg_factor = reg_factor_widget.value
    solver_params = {}
    for widget in solver_specific_params_widget.children:
        val = widget.value
        if val == "None": val = None
        elif val == "True" or val == "true": val = True
        elif val == "False" or val == "False": val = False
        elif "{" in val and "}" in val: val = json.loads(val)
        else:
            try:
                val = float(val)
            except:
                pass
        solver_params[widget.description] = val
    with show_loading():
        res = inversion(initial_model_id, z0, reg_factor, method, tool, solver_params)
        args[0].result = res
        args[0].settings = {"initial_model":initial_model_id, "reg_factor":reg_factor, 
                            "method":method, "tool":tool, "solver_params":solver_params}
run_widget.on_click(button_on_click)

label = widgets.Label('Ready')
@contextmanager
def show_loading():
    label.value = 'Running...'
    yield
    label.value = 'Ready'

w = widgets.VBox(children=[method_widget, tool_widget, initial_model_widget, z0_widget, reg_factor_widget,
                          solver_specific_params_widget, run_widget, label])
display(w)

In [6]:
#####################################################################################
#                                                                                   #
#    Uncomment below after you've selected parameters and run an inversion above    #
#                                                                                   #
#####################################################################################

# inv_res = run_widget.result
# plot_from_result(inv_res)
# inv_res.summary()

---

## Watermark

In [7]:
watermark_list = ["cofi", "numpy", "scipy", "matplotlib", "emcee", "arviz", "ipywidgets"]
for pkg in watermark_list:
    pkg_var = __import__(pkg)
    print(pkg, getattr(pkg_var, "__version__"))

cofi 0.1.2.dev6
numpy 1.21.6
scipy 1.8.1
matplotlib 3.5.2
emcee 3.1.2
arviz 0.12.1
ipywidgets 7.7.0
